In [1]:
# the following copied from https://github.com/openmm/openmmforcefields

# Create an OpenFF Molecule object for benzene from SMILES
from openff.toolkit.topology import Molecule
molecule = Molecule.from_smiles('CC(=O)[O-]')

# Create the GAFF template generator
from openmmforcefields.generators import GAFFTemplateGenerator
gaff = GAFFTemplateGenerator(molecules=molecule)

# Create an OpenMM ForceField object with AMBER ff14SB and TIP3P with compatible ions
from simtk.openmm.app import ForceField
forcefield = ForceField('amber/ff14SB.xml', 'amber/tip3p_standard.xml')

# Register the GAFF template generator
forcefield.registerTemplateGenerator(gaff.generator)

In [4]:
# You can now parameterize an OpenMM Topology object that contains the specified molecule.
# forcefield will load the appropriate GAFF parameters when needed, and antechamber
# will be used to generate small molecule parameters on the fly.
from simtk.openmm.app import PDBFile
pdbfile = PDBFile('mixture.pdb')

#system = forcefield.createSystem(pdbfile.topology)

In [5]:
# initialize the MD system with gaff-parametrized forcefield + amber ions ff
from mdtools import MDSystem
from simtk.unit import *

mdsystem = MDSystem(pdbfile.topology, pdbfile.positions, forcefield)
mdsystem.buildSimulation(posre=True)

In [ ]:
mdsystem.equilibrate(100*picoseconds)

In [ ]:
mdsystem.save('equilibrated.pdb')

In [ ]:
# NPT equilibration, to get the correct volume
mdsystem.buildSimulation(filePrefix=f"NPT_run", ensemble='NPT',
                         saveStateData=True, stateDataInterval=500)
mdsystem.simulate(1000*picoseconds)
mdsystem.save('post_NPT.pdb')

In [ ]:
# NVT production run, no E field
pdbfile = PDBFile('post_NPT.pdb')
mdsystem = MDSystem(pdbfile.topology, pdbfile.positions, forcefield)
mdsystem.buildSimulation(filePrefix=f"NVT_run", ensemble='NVT',
                         saveTrajectory=True, trajInterval=500, saveVelocities=True,
                         saveStateData=True, stateDataInterval=500)
mdsystem.simulate(1000*picoseconds)
mdsystem.save('post_NVT.pdb')

In [ ]:
import mdtraj
import numpy as np

traj = mdtraj.load('NVT_run.h5')
traj[::20].save('NVT_movie.pdb')

In [ ]:
for reporter in mdsystem.simulation.reporters:
    try:
        reporter.close()
    except:
        continue

In [ ]:
# NVT production run, E field at 1MV/cm
field_strength = 1e8 # V/m
pdbfile = PDBFile('post_NPT.pdb')
mdsystem = MDSystem(pdbfile.topology, pdbfile.positions, forcefield)
mdsystem.buildSimulation(filePrefix=f"NVT_EF_run", ensemble='NVT',
                         efx=True, ef=(field_strength, 0, 0),
                         saveTrajectory=True, trajInterval=500, saveVelocities=True,
                         saveStateData=True, stateDataInterval=500)
mdsystem.simulate(1000*picoseconds)
mdsystem.save('NVT_EF_run.pdb')